In [1]:
#Most of the blockchain-creation code was from 
# https://dzone.com/articles/how-to-create-your-own-cryptocurrency-blockchain-i 
#and edited to fit my idea, it is not a real and functional blockchain as there is no proof-of-work algorithm. 
#The code that isn't the Block/BlockChain classes is dominantly my own work. 

In [1]:
#RUN THIS FIRST, ONCE

import hashlib
import time
import pandas as pd

class Block(object):

    def __init__(self, index, proof_number, previous_hash, data, timestamp=None):
        self.index = index
        self.proof_number = proof_number
        self.previous_hash = previous_hash
        self.data = data
        self.timestamp = timestamp or time.time()

    @property
    def compute_hash(self):
        string_block = "{}{}{}{}{}".format(self.index, self.proof_number, self.previous_hash, self.data, self.timestamp)
        return hashlib.sha256(string_block.encode()).hexdigest()

    def __repr__(self):
        return "{} - {} - {} - {} - {}".format(self.index, self.proof_number, self.previous_hash, self.data, self.timestamp)

class BlockChain(object):

    def __init__(self):
        self.chain = []
        self.current_data = []
        self.nodes = set()
        self.build_genesis()

    def build_genesis(self):
        self.build_block(proof_number=0, previous_hash=0)

    def build_block(self, proof_number, previous_hash):
        block = Block(
            index=len(self.chain),
            proof_number=proof_number,
            previous_hash=previous_hash,
            data=self.current_data
        )

        self.current_data = []
        self.chain.append(block)
        return block

    
    @staticmethod
    def confirm_validity(block, previous_block):

        if previous_block.index + 1 != block.index:
            return False

        elif previous_block.compute_hash != block.previous_hash:
            return False

        elif block.timestamp != previous_block.timestamp:
            return False

        return True

    def get_data(self, data, sender, receiver, amount, previous_hash, block):
        self.current_data.append({
            'data': data,
            'sender': sender,
            'receiver': receiver,
            'amount': amount,
            'previous_hash' : previous_hash,
            'block' : block
        })
        return True        
    
    
    @staticmethod
    def proof_of_work(last_proof):
        pass

    @property
    def latest_block(self):
        return self.chain[-1]

    def chain_validity(self):
        pass        

    def block_mining(self, details_miner):       
        self.get_data(
            data={},
            sender="0", #it implies that this node has created a new block
            receiver=details_miner,
            amount=1, #creating a new block (or identifying the proof number) is awared with 1
            previous_hash=self.latest_block.compute_hash,
            block=self.latest_block
        )
        last_block = self.latest_block
        last_proof_number = last_block.proof_number
        proof_number = self.proof_of_work(last_proof_number)
        last_hash = last_block.compute_hash
        block = self.build_block(proof_number, last_hash)
        return vars(block)  

    def create_node(self, address):
        self.nodes.add(address)
        return True

    @staticmethod
    def get_block_object(block_data):        
        return Block(
            block_data['index'],
            block_data['proof_number'],
            block_data['previous_hash'],
            block_data['data'],
            timestamp=block_data['timestamp']
        )



In [2]:
#RUN THIS ONCE

blockchain = BlockChain()

last_block = blockchain.latest_block

last_proof_number = last_block.proof_number

proof_number = blockchain.proof_of_work(last_proof_number)



In [24]:
#RUN THIS MULTIPLE TIMES WITH DIFFERENT FILENAMES TO SIMULATE CHANGING DATA

import pandas as pd

file = input("Filename, case sensitive: ")
students = pd.read_csv (file, index_col="id")
print(students)
students=pd.DataFrame(students)

Filename, case sensitive: class2012.csv
                 last      first   year   month  degree      major
id                                                                
36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
98765            Wash      Hoban   2012      12     PhD      pilot
56565           Obama     Barack   2012      12     PhD   politics


In [25]:
#RUN THIS MULTIPLE TIMES AFTER RUNNING THE ABOVE TO SIMULATE CHANGING DATA

print("STARTING")
print(blockchain.chain)

blockchain.get_data(
    data = students,
    sender="0", 
    receiver="Blockchain Bootcamp", 
    amount=1,
    previous_hash = last_block.compute_hash,
    block = blockchain.build_block(proof_number, last_proof_number))
    
print("CREATION SUCCESSFUL")
print(blockchain.chain)

STARTING
[0 - 0 - 0 - [] - 1608595262.5113883, 1 - None - 0 - [] - 1608595270.9434788, 2 - None - 0 - [{'data':                  last      first   year   month  degree      major
id                                                                
36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
98765            Wash      Hoban   2012      12     PhD      pilot
56565           Obama     Barack   2012      12     PhD   politics, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': 'eb5f36bede48010d9a75973f09240d00df4eceadb5c269b5d83e3bf4d2382169', 'block': 1 - None - 0 - [] - 1608595270.9434788}] - 1608595808.5674455, 3 - None - 0 - [{'data':         last    first   year   month  degree     major
id                                                    
66666   Name   Holder   2019       7     Bsc   Physics
98989   Test   Person   2019       6     PhD   Physics, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': 'eb5f3

In [26]:
#This is just a visual to show what's currently in the chain, that the hashes exist and it's working as intended
#This isn't something the general users would get to see

print(blockchain.current_data, "\n\n")
print(len(blockchain.chain)-1)
for x in range(len(blockchain.chain)-1):
    print(x, ":\n", blockchain.chain[x])

[{'data':                  last      first   year   month  degree      major
id                                                                
36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
98765            Wash      Hoban   2012      12     PhD      pilot
56565           Obama     Barack   2012      12     PhD   politics, 'sender': '0', 'receiver': 'Blockchain Bootcamp', 'amount': 1, 'previous_hash': 'eb5f36bede48010d9a75973f09240d00df4eceadb5c269b5d83e3bf4d2382169', 'block': 5 - None - 0 - [] - 1608595867.0765767}] 


5
0 :
 0 - 0 - 0 - [] - 1608595262.5113883
1 :
 1 - None - 0 - [] - 1608595270.9434788
2 :
 2 - None - 0 - [{'data':                  last      first   year   month  degree      major
id                                                                
36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
98765            Wash      Hoban   2012      12     PhD      pilot
56565           Obama     Barack   2012      12     PhD   politics, 

In [30]:
#This gives you a csv record of all previous graduates


output=pd.DataFrame()
for x in range(len(blockchain.chain)-1):
    if (len(blockchain.chain[x].data) >= 1):
        output = output.append(blockchain.chain[x].data[0].get("data"))
output = output.append(blockchain.current_data[0].get("data"))

print(output)
with open('Graduates.txt', 'a') as f:
    f.write(
        output.to_string(header = True, index = True)
    )
#output.to_csv("Graduates.csv")

                 last            first   year   month  degree            major
id                                                                            
36456   Cucumberpatch         Bunnyboy   2012       7     Bsc           acting
98765            Wash            Hoban   2012      12     PhD            pilot
56565           Obama           Barack   2012      12     PhD         politics
66666            Name           Holder   2019       7     Bsc          Physics
98989            Test           Person   2019       6     PhD          Physics
12345            Abel           Tasman   2020      12     Bsc        astronomy
54321            Cook          Captain   2020      12     Bsc          weather
10001         Hillary       Sir Edmund   2020       7     PhD   mountaineering
10000           Putin         Vladimir   2020      12     PhD             math
10101          Sherpa   Tenzing Norgay   2020       7     PhD   mountaineering
77777         Someone             Here   2020      1

In [29]:
#This is what gives you the data in a useable way

for x in range(len(blockchain.chain)-1):
    #print(blockchain.chain[x].data)
    if (len(blockchain.chain[x].data) >= 1):
        print(blockchain.chain[x].data[0].get("data"))
        
print("\n\nCurrent Data Block:\n", blockchain.current_data[0].get("data"))

                 last      first   year   month  degree      major
id                                                                
36456   Cucumberpatch   Bunnyboy   2012       7     Bsc     acting
98765            Wash      Hoban   2012      12     PhD      pilot
56565           Obama     Barack   2012      12     PhD   politics
        last    first   year   month  degree     major
id                                                    
66666   Name   Holder   2019       7     Bsc   Physics
98989   Test   Person   2019       6     PhD   Physics
           last            first   year   month  degree            major
id                                                                      
12345      Abel           Tasman   2020      12     Bsc        astronomy
54321      Cook          Captain   2020      12     Bsc          weather
10001   Hillary       Sir Edmund   2020       7     PhD   mountaineering
10000     Putin         Vladimir   2020      12     PhD             math
10101  

In [22]:
#Theretically works to mine the data, but without the proof-of-work it just kinda... gets the data. easily.
blockchain.block_mining("Blockchain Bootcamp")

{'index': 4,
 'proof_number': None,
 'previous_hash': '84eb007960c71929e4710c5c7de95e7dda18e711a061284e13cb1caf8839a0da',
 'data': [{'data':            last            first   year   month  degree            major
   id                                                                      
   12345      Abel           Tasman   2020      12     Bsc        astronomy
   54321      Cook          Captain   2020      12     Bsc          weather
   10001   Hillary       Sir Edmund   2020       7     PhD   mountaineering
   10000     Putin         Vladimir   2020      12     PhD             math
   10101    Sherpa   Tenzing Norgay   2020       7     PhD   mountaineering
   77777   Someone             Here   2020      10     Bsc          cooking,
   'sender': '0',
   'receiver': 'Blockchain Bootcamp',
   'amount': 1,
   'previous_hash': 'eb5f36bede48010d9a75973f09240d00df4eceadb5c269b5d83e3bf4d2382169',
   'block': 3 - None - 0 - [{'data':         last    first   year   month  degree     major
 